## 0.  Re-creating the sqlite db to get rid of "chunk_index" as primary key

In [46]:
import sqlite3

def create_chunk_db(img_chunks: list, db_filepath: str) -> None:
    # Create a lookup table for chunks
    conn = sqlite3.connect(db_filepath)
    cursor = conn.cursor()

    # Create the table  - delete old table if it exists
    cursor.execute("DROP TABLE IF EXISTS chunks_reverse_lookup")

    # Create the table - alias rowid as chunk_index
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS chunks_reverse_lookup (
            img_path TEXT,
            chunk TEXT
        );
    """)

    # Insert data into the table
    for chunk_index, entry in enumerate(img_chunks):
        img_path = entry["img_path"]
        chunk = entry["chunk"]
        cursor.execute(
            "INSERT INTO chunks_reverse_lookup (img_path, chunk) VALUES (?, ?)",
            (img_path, chunk),
        )

    conn.commit()
    conn.close()
    

def query_all(db_filepath: str) -> list:
    conn = sqlite3.connect(db_filepath)
    cursor = conn.cursor()
    query = f"SELECT rowid, * FROM chunks_reverse_lookup"
    cursor.execute(query)
    rows = cursor.fetchall()
    rows = [{"index": row[0], "img_path": row[1], "chunk": row[2]} for row in rows]
    return rows
    
    
def query_sqlite_db(indices: list, db_filepath: str) -> list:
    conn = sqlite3.connect(db_filepath)
    cursor = conn.cursor()
    query = f"SELECT rowid, * FROM chunks_reverse_lookup WHERE rowid IN {tuple(indices)}"
    cursor.execute(query)
    rows = cursor.fetchall()
    rows = [{"index": row[0], "img_path": row[1], "chunk": row[2]} for row in rows]
    rows = sorted(rows, key=lambda x: indices.index(x["index"]))  # re-sort rows according to input indices
    for row in rows:
        query = f"SELECT rowid, * FROM chunks_reverse_lookup WHERE rowid=(SELECT MIN(rowid) FROM chunks_reverse_lookup WHERE img_path='{row['img_path']}')"
        cursor.execute(query)
        full_description_row = cursor.fetchall()
        row["full_description"] = full_description_row[0][2]
    conn.close()
    return rows


def collect_removal_rowids(old_imgs_to_be_removed: list, db_path: str) -> list:
    try:
        if len(old_imgs_to_be_removed) > 0:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()
            query = f"""SELECT rowid FROM chunks_reverse_lookup WHERE img_path IN ({','.join(['"'+v+'"' for v in old_imgs_to_be_removed])})"""
            cursor.execute(query)
            rows = cursor.fetchall()
            rowids = [v[0] for v in rows]
            conn.close()
            return rowids
        else:
            return []
    except Exception as e:
        raise ValueError("FAILURE: collect_removal_rowids failed with exception{e}")
    
    
def delete_removal_rowids_from_reverse_lookup(rowids: list, db_path: str) -> None:
    try:
        if len(rowids) > 0:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()
            if len(rowids) == 1:
                query =  f"""DELETE FROM chunks_reverse_lookup WHERE rowid IN ({str(rowids[0])})"""
            else:
                query =  f"""DELETE FROM chunks_reverse_lookup WHERE rowid IN ({','.join([str(v) for v in rowids])})"""
            cursor.execute(query)
            conn.commit()
            conn.close()

            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()
            cursor.execute("VACUUM;")
            conn.commit()
            conn.close()
    except Exception as e:
        raise ValueError("FAILURE: delete_removal_rowids failed with exception{e}")

In [30]:
from meme_search.utilities.text_extraction import extract_text_from_imgs
from meme_search.utilities.chunks import create_all_img_chunks
from meme_search.utilities.imgs import collect_img_paths

all_img_paths = [collect_img_paths()[0], collect_img_paths()[1], collect_img_paths()[2]]


STARTING: collect_img_paths
SUCCESS: collect_img_paths ran successfully - image paths loaded from '/Users/jeremywatt/Desktop/meme_search/data/input'
STARTING: collect_img_paths
SUCCESS: collect_img_paths ran successfully - image paths loaded from '/Users/jeremywatt/Desktop/meme_search/data/input'
STARTING: collect_img_paths
SUCCESS: collect_img_paths ran successfully - image paths loaded from '/Users/jeremywatt/Desktop/meme_search/data/input'


In [31]:
moondream_answers = extract_text_from_imgs(all_img_paths)
img_chunks = create_all_img_chunks(all_img_paths, moondream_answers)
create_chunk_db(img_chunks, "test.db")

STARTING: extract_text_from_imgs
INFO: prompting moondream for a description of image: './data/input/test_meme_1.jpg'
DONE!
INFO: prompting moondream for a description of image: './data/input/test_meme_2.jpg'
DONE!
INFO: prompting moondream for a description of image: './data/input/test_meme_3.jpg'
DONE!
SUCCESS: extract_text_from_imgs succeeded
STARTING: create_all_img_chunks
SUCCESS: create_all_img_chunks ran successfully


In [35]:
rowids = collect_removal_rowids(["./data/input/test_meme_2.jpg"], "test.db")

In [36]:
rowids[0]

34

In [47]:
delete_removal_rowids_from_reverse_lookup(rowids, "test.db")

In [48]:
query_all("test.db")

[{'index': 1,
  'img_path': './data/input/test_meme_1.jpg',
  'chunk': 'the image features a man dressed in a suit and tie, holding a glass of wine in his right hand. he is wearing a white flower on his lapel. the background is blurred, suggesting a room with a window and a painting on the wall. the text in the image reads, "when you see there\'s a moderator rulebanning not safe for work content but you\'reunemployed."'},
 {'index': 2,
  'img_path': './data/input/test_meme_1.jpg',
  'chunk': 'the image features a'},
 {'index': 3,
  'img_path': './data/input/test_meme_1.jpg',
  'chunk': 'features a man dressed'},
 {'index': 4,
  'img_path': './data/input/test_meme_1.jpg',
  'chunk': 'man dressed in a'},
 {'index': 5,
  'img_path': './data/input/test_meme_1.jpg',
  'chunk': 'in a suit and'},
 {'index': 6,
  'img_path': './data/input/test_meme_1.jpg',
  'chunk': 'suit and tie, holding'},
 {'index': 7,
  'img_path': './data/input/test_meme_1.jpg',
  'chunk': 'tie, holding a glass'},
 {'ind

In [27]:
conn = sqlite3.connect("test.db")
cursor = conn.cursor()
cursor.execute("VACUUM;")
conn.commit()
conn.close()

## 1. Collect names from directory and db for compare

In [1]:
from meme_search.utilities.imgs import collect_img_paths
import sqlite3
from meme_search import sqlite_db_path

def get_current_indexed_img_names():
    try:
        print("STARTING: collecting currently indexed names")
        conn = sqlite3.connect(sqlite_db_path)
        cursor = conn.cursor()
        query = f"SELECT DISTINCT(img_path) FROM chunks_reverse_lookup"
        cursor.execute(query)
        rows = cursor.fetchall()
        rows = [v[0] for v in rows]
        conn.close()
        print("SUCCESS: get_current_indexed_img_names ran successfully")
        return rows
    except Exception as e:
        raise ValueError(f"FAILURE: get_current_indexed_img_names failed with exception {e}")


all_img_paths = collect_img_paths()
all_img_paths_stubs = ["./" + "/".join(v.split("/")[-3:]).strip() for v in all_img_paths]
current_indexed_names = get_current_indexed_img_names()

new_imgs_to_be_indexed = list(set(all_img_paths_stubs) - set(current_indexed_names))
old_imgs_to_be_removed = list(set(current_indexed_names) - set(all_img_paths_stubs))

/Users/jeremywatt/Desktop/meme_search/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


STARTING: collect_img_paths
SUCCESS: collect_img_paths ran successfully - image paths loaded from '/Users/jeremywatt/Desktop/meme_search/data/input'
STARTING: collecting currently indexed names
SUCCESS: get_current_indexed_img_names ran successfully


## 2.  Remove any db trace of img not in directory anymore

In [36]:
import numpy as np
import faiss


def collect_removal_rowids(old_imgs_to_be_removed: list) -> list:
    try:
        if len(old_imgs_to_be_removed) > 0:
            conn = sqlite3.connect(sqlite_db_path)
            cursor = conn.cursor()
            query = f"""SELECT rowid FROM chunks_reverse_lookup WHERE img_path IN ({','.join(['"'+v+'"' for v in old_imgs_to_be_removed])})"""
            cursor.execute(query)
            rows = cursor.fetchall()
            rowids = [v[0] for v in rows]
            conn.close()
            return rowids
        else:
            return []
    except Exception as e:
        raise ValueError("FAILURE: collect_removal_rowids failed with exception{e}")
    
    
def delete_removal_rowids_from_reverse_lookup(rowids: list) -> None:
    try:
        if len(rowids) > 0:
            conn = sqlite3.connect(sqlite_db_path)
            cursor = conn.cursor()
            query =  f"""DELETE FROM chunks_reverse_lookup WHERE rowid IN ({','.join([str(v) for v in rowids])})"""
            cursor.execute(query)
            rows = cursor.fetchall()
            rowids = [v[0] for v in rows]
            conn.close()
    except Exception as e:
        raise ValueError("FAILURE: delete_removal_rowids failed with exception{e}")


def delete_removal_rowids_from_vector_db(rowids: list, 
                                         path_to_index: str) -> None:
    try:
        if len(rowids) > 0:
            index = faiss.read_index(path_to_index)
            remove_set = np.array(rowids, dtype=np.int64)
            index.remove_ids(remove_set)
            faiss.write_index(index, path_to_index)
    except Exception as e:
        raise ValueError("FAILURE: delete_removal_rowids failed with exception{e}")

In [35]:
test_paths = [all_img_paths_stubs[0]]
print(test_paths)

rowids = collect_removal_rowids(test_paths)
print(rowids)

delete_removal_rowids_from_reverse_lookup(rowids)


['./data/input/test_meme_1.jpg']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]


## 3. Adding new additions

In [29]:
# 



In [30]:
rowids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32]

'DELETE FROM chunks_reverse_lookup WHERE rowid IN (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32)'

In [2]:
new_imgs_to_be_indexed = [v for v in all_img_paths_names if v in 

In [3]:
old_imgs_to_be_removed

[]

In [4]:
new_imgs_to_be_indexed

[]

In [19]:
rows[0][0].split("/")[-1]

'test_meme_1.jpg'

In [21]:
rows

['test_meme_1.jpg',
 'test_meme_2.jpg',
 'test_meme_3.jpg',
 'test_meme_4.jpg',
 'test_meme_5.jpg',
 'test_meme_6.jpg',
 'test_meme_7.jpg',
 'test_meme_8.jpg',
 'test_meme_9.jpg']